# Clusterer

## Description : 

Foor this part, I will use the `distilcamembert-base-nli` model from CMArkea to perform clustering of my reviews. The model uses zero-shot classification to assign labels to the reviews based on their content.

I will use the OpenAI model to induce a finite set of labels for the reviews and the zero-shot model will assign them to the reviews. TO help me prompt the OpenAI model, I will use a random sample of 100 reviews.

Once I have the labels, I will use the `distilcamembert-base-nli` model to perform clustering.

## Setup

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
MONGODB_URI = os.getenv('MONGODB_URI')

## Import Dataset

In [2]:
import pandas as pd

file_path = "../data/catering_reviews.csv"
data = pd.read_csv(file_path)

if data is not None:
    print("Processed Data:")
    print(data.head())
else:
    print("No data to process.")

Processed Data:
     author  rating                                               text  \
0   Estelle     5.0  Monsieur M a surpassé toutes nos attentes lors...   
1  Patricia     5.0  Nous recommandons vivement les services de Mon...   
2     Julie     4.8  Un grand merci à Camille et à toute l’équipe d...   
3  Amandine     5.0  L’équipe était très professionnel, souriante e...   
4  Mathilde     5.0  Accompagnement en amont, prestation le jour J,...   

         date sentiment        source                company_id company_name  
0  2024-04-11  positive  mariages.net  6852d3d474c91954046c1cdb   Monsieur M  
1  2025-05-25  positive  mariages.net  6852d3d474c91954046c1cdb   Monsieur M  
2  2024-12-10  positive  mariages.net  6852d3d474c91954046c1cdb   Monsieur M  
3  2024-11-09  positive  mariages.net  6852d3d474c91954046c1cdb   Monsieur M  
4  2025-02-02  positive  mariages.net  6852d3d474c91954046c1cdb   Monsieur M  


## Preprocessing

In [3]:
# only need the text
df = data[["text"]].dropna() 

# Sample the data to 100 rows
df_sample = df.sample(n=100, random_state=42).reset_index(drop=True)

# Display the first few rows of the data
print(df_sample.head())
print(df.head())

                                                text
0  Toutes les assiettes sont revenues vides, c'ét...
1  Nous sommes très satisfaits de notre repas de ...
2  Tout était parfait, le service impeccable et l...
3  Mariés, témoins, invités... Nous avons tous ét...
4  Nous avons choisi L’instant Réception pour not...
                                                text
0  Monsieur M a surpassé toutes nos attentes lors...
1  Nous recommandons vivement les services de Mon...
2  Un grand merci à Camille et à toute l’équipe d...
3  L’équipe était très professionnel, souriante e...
4  Accompagnement en amont, prestation le jour J,...


## Prepare the prompt

In [4]:
from openai import OpenAI

# Create the client with the API key
client = OpenAI(api_key=OPENAI_API_KEY)

In [5]:
# Create the prompt function
def suggest_labels(prompt, model="gpt-3.5-turbo"): # Andrew mentioned that the prompt/ completion paradigm is preferable for this class
  messages = [{"role": "user", "content": prompt}]
  response = client.chat.completions.create(
      model=model,
      messages=messages,
      temperature=0.3,
  )
  return response.choices[0].message.content

In [6]:
sample_reviews = ""

for i, row in df_sample.iterrows():
    sample_reviews += f"Avis {i+1}: {row['text']}\n\n"

print("Sample Reviews:")
print(sample_reviews)

Sample Reviews:
Avis 1: Toutes les assiettes sont revenues vides, c'était un régal !
Tous nos invités nous ont félicité du traiteur.
Personnel agréable et disponible.
Équipe à l'écoute des mariés qui ont respecté notre rythme, merci.

Avis 2: Nous sommes très satisfaits de notre repas de mariage. Tout s'est parfaitement enchaîné, le service est rapide (tout le monde mange chaud !), tout est de bonne qualité... Bref, tout est réuni pour passer une bonne soirée ! La maître d'hôtel est discrète mais s'assure toujours que tout se passe bien, ce qui est fort appréciable. Nous ne pouvons que recommander Traiteur Calvados !

Avis 3: Tout était parfait, le service impeccable et la cuisine délicieuse. Merci beaucoup.

Avis 4: Mariés, témoins, invités... Nous avons tous été conquis tant par la qualité du vin d'honneur et du repas que par le service ! Nos témoins ont également souligné la réactivité et l'adaptabilité de l'ensemble de l'équipe notamment pour permettre les animations. Bref tout a é

In [17]:
prompt = f"""Tu es un expert en clustering d'avis de traiteurs de mariage. \
  Voici des avis de clients sur des traiteurs pour des mariages :\n\n{sample_reviews}\n\n \
  Ta tâche est de proposer une liste de 10 catégories thématiques pour classer ou regrouperdes avis de clients sur des traiteurs pour des mariages. \
  Donne uniquement une liste de mots ou expressions, sous forme de liste Python. Exemple : ["nourriture", "service", "ambiance", "ponctualité", "prix", "professionnalisme", "présentation"] etc. \
  Ne donne pas d'explications, juste la liste. \
  """

# Function call
suggested_labels = suggest_labels(prompt)
print("Suggested Labels:")
print(suggested_labels)

Suggested Labels:
["qualité des plats", "service client", "professionnalisme de l'équipe", "présentation des plats", "flexibilité", "rapport qualité/prix", "communication", "adaptabilité", "satisfaction des invités", "expérience globale"]


## DistilCamemBERT Zero-shot classification test

In [18]:
from transformers import pipeline

classifier = pipeline(
    task="zero-shot-classification",
    model="cmarkea/distilcamembert-base-nli",
    tokenizer="cmarkea/distilcamembert-base-nli"
)

Device set to use mps:0


In [21]:
review_text = df_sample["text"].tolist()[0] # Test avec 1

# print("suggested_labels", suggested_labels)

# result = classifier(
#     sequences=review_text,
#     candidate_labels=suggested_labels,
#     hypothesis_template="Ce commentaire concerne {}."
# )

# print("Classification Result:")
# print(result)

def classify_themes(text, classifier=classifier, labels=suggested_labels, top_k=3):
    result = classifier(sequences=text, candidate_labels=labels, hypothesis_template="Cet avis concerne {}.")
    return result["labels"][:top_k]

df["clusters"] = df["text"].apply(lambda x: classify_themes(x))

In [22]:
df.head()

,text,clusters
0,Monsieur M a surpassé toutes nos attentes lors...,"[""professionnalisme de l'équipe"", ""présentatio..."
1,Nous recommandons vivement les services de Mon...,"[""rapport qualité/prix"", ""service client"", ""ad..."
2,Un grand merci à Camille et à toute l’équipe d...,"[""professionnalisme de l'équipe"", ""communicati..."
3,"L’équipe était très professionnel, souriante e...","[""professionnalisme de l'équipe"", ""communicati..."
4,"Accompagnement en amont, prestation le jour J,...","[""adaptabilité"", ""flexibilité"", ""communication""]"


In [20]:
df_result = pd.DataFrame({
    "label": result["labels"],
    "score": result["scores"]
})

df_result["score"] = df_result["score"].round(3)
df_result.sort_values(by="score", ascending=False, inplace=True)
print(df_result)

                             label  score
0  "professionnalisme de l'équipe"  0.195
1                  "communication"  0.165
2                 "service client"  0.147
3         "présentation des plats"  0.120
4                   "adaptabilité"  0.090
5           "rapport qualité/prix"  0.085
6             ["qualité des plats"  0.069
7                    "flexibilité"  0.062
8            "expérience globale"]  0.042
9       "satisfaction des invités"  0.025
